##### Configurações Iniciais

In [11]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np

from sklearn.metrics import precision_score, recall_score, f1_score, ConfusionMatrixDisplay

from Modules.operation_analiser import OperationLoader, OperationAnalyzer
from Modules.forecast_loader import ForecastLoader
from Modules.variables_info import variables_info
from Modules.scenario_evaluator import RainScenarioModel

In [12]:
# Parâmetros gerais
lat_porto = -25.500511
lon_porto = -48.519331
forecast_path = '../Database/forecast_data'
paradas_path = '../Database/Parada_2024_chuva.xlsx'
timebox = ('202405', '202412')

variaveis = variables_info['variaveis']
legendas = variables_info['legendas']
legendas_dict = dict(zip(variaveis, legendas))

# Repositorios de Saída
base_output = '../Output/Analise_02'
os.makedirs('../Output/Analise_02', exist_ok=True)
for subfolder in ['Tables', 'Imagens', 'Imagens/Thresholds', 'Imagens/Modelo_01/Metricas', 'Imagens/Modelo_01/Serie_temporais', 'Imagens/Modelo_02/Metricas', 'Imagens/Modelo_02/Serie_temporais']:
    os.makedirs(os.path.join(base_output, subfolder), exist_ok=True)

##### Importação dos Dados para Analise

In [13]:
forecast = ForecastLoader(forecast_path, lat_target=lat_porto, lon_target=lon_porto)
df_forecast = forecast.load_forecasts(timebox, drop_duplicates=True)

operation_loader = OperationLoader(paradas_path)
df_paradas = operation_loader.load()

analyzer = OperationAnalyzer(df_paradas, df_forecast)
df_unificado = analyzer.marcar_paradas()

In [14]:
stats = analyzer.analisar_variaveis(variaveis, export_path='../Output/Analise_02/Tables/estatisticas_por_classe.csv')

##### Analises

###### Avaliação Individual das Variaveis para Previsão de Evento de Parada

In [15]:
resultados_thresholds = []

for var, l in zip(variaveis, legendas):
    y_true = df_unificado['parada']
    metricas_var = []

    if var.startswith('lwe_'):
        thresholds = [round(x, 2) for x in list(np.arange(0, 5.01, 0.1))]
    elif var.startswith('precipitation_probability'):
        thresholds = [round(x, 1) for x in range(0, 101, 5)]
    else:
        continue

    for thresh in thresholds:
        y_pred = (df_unificado[var] > thresh).astype(int)

        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)

        metricas_var.append({
            'variavel': var,
            'threshold': thresh,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        })

    df_m = pd.DataFrame(metricas_var)
    df_m.to_csv(f'{base_output}/Tables/thresholds_{var}.csv', index=False)

    plt.figure(figsize=(10, 4))
    plt.plot(df_m['threshold'], df_m['f1_score'], marker='o')
    plt.title(f'{l}')
    plt.xlabel('Threshold')
    plt.ylabel('F1 Score')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{base_output}/Imagens/Thresholds/{var}_f1_curve.png')
    plt.close()

    melhor = df_m.sort_values('f1_score', ascending=False).iloc[0].to_dict()
    resultados_thresholds.append(melhor)

best_thresholds_df = pd.DataFrame(resultados_thresholds)
best_thresholds_df = best_thresholds_df.sort_values('f1_score', ascending=False)
best_thresholds_df.to_csv(f'{base_output}/Tables/melhores_thresholds.csv', index=False)

print('Melhores thresholds por variável (baseado em F1 Score):')
display(best_thresholds_df)

Melhores thresholds por variável (baseado em F1 Score):


,variavel,threshold,precision,recall,f1_score
12,precipitation_probability_55km,25.0,0.635674,0.720430,0.675403
9,precipitation_probability_35km,25.0,0.625078,0.714695,0.666890
4,lwe_precipitation_smooth_rate_maximum_20km,0.4,0.638255,0.681720,0.659272
8,lwe_precipitation_smooth_rate_maximum_35km,0.5,0.547317,0.804301,0.651379
6,precipitation_probability_20km,15.0,0.604824,0.701075,0.649402
11,lwe_precipitation_smooth_rate_maximum_55km,1.1,0.580980,0.722581,0.644089
2,lwe_precipitation_smooth_rate_maximum_10km,0.0,0.588756,0.668100,0.625923
3,precipitation_probability_10km,0.0,0.588756,0.668100,0.625923
0,lwe_precipitation_smooth_rate,0.0,0.690710,0.453047,0.547186
1,lwe_precipitation_smooth_rate_minimum_10km,0.0,0.811441,0.274552,0.410284


##### Modelo 01
###### Avaliação das Variaveis para Avaliação de Evento de Parada Baseado em Diferentes Métricas

In [16]:
cenarios = {
    'conservador': best_thresholds_df.sort_values('recall', ascending=False).head(3),
    'moderado': best_thresholds_df.sort_values('f1_score', ascending=False).head(3),
    'arrojado': best_thresholds_df.sort_values('precision', ascending=False).head(3),
}

modelador = RainScenarioModel(
    df=df_unificado,
    cenarios=cenarios,
    base_output=base_output,
    legendas_dict=legendas_dict,
    modelo_nome="Modelo_01"  # ou "Modelo_01", "Modelo_RF", etc.
)

modelador.aplicar_cenarios()


Cenário: CONSERVADOR
Variáveis usadas:
[('lwe_precipitation_smooth_rate_maximum_35km', 0.5), ('lwe_precipitation_smooth_rate_maximum_55km', 1.1), ('precipitation_probability_55km', 25.0)]
Precision: 0.65 | Recall: 0.67 | F1 Score: 0.66

Cenário: MODERADO
Variáveis usadas:
[('precipitation_probability_55km', 25.0), ('precipitation_probability_35km', 25.0), ('lwe_precipitation_smooth_rate_maximum_20km', 0.4)]
Precision: 0.67 | Recall: 0.64 | F1 Score: 0.65

Cenário: ARROJADO
Variáveis usadas:
[('lwe_precipitation_smooth_rate_minimum_35km', 0.0), ('lwe_precipitation_smooth_rate_minimum_55km', 0.0), ('lwe_precipitation_smooth_rate_minimum_20km', 0.0)]
Precision: 0.91 | Recall: 0.04 | F1 Score: 0.08


##### Modelo 02
###### Analise Combinada das Variaveis (Estatistica + Metricas) para Avaliação de Evento de Parada

In [17]:
estatisticas = pd.read_csv('../Output/Analise_02/Tables/estatisticas_por_classe.csv')
thresholds = pd.read_csv('../Output/Analise_02/Tables/melhores_thresholds.csv')

estatisticas['legenda'] = estatisticas['variavel'].map(legendas_dict)
thresholds['legenda'] = thresholds['variavel'].map(legendas_dict)

In [18]:
# Ordenar variáveis pela diferença de média
ranking_dif = estatisticas[['variavel', 'legenda', 'diferenca_media']].sort_values(by='diferenca_media', ascending=False)

# Visualizar top 10 variáveis com maior diferença
plt.figure(figsize=(10,6))
sns.barplot(data=ranking_dif.head(10), y='legenda', x='diferenca_media', palette='Blues_d')
plt.title('Top 10 Variáveis com Maior Diferença de Média (Parada vs Não Parada)')
plt.xlabel('Diferença de Média')
plt.ylabel(None)
plt.grid(True, axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig(f'{base_output}/Imagens/Modelo_02/top10_diferenca_media.png')
plt.close()

# Cruzar com desempenho dos thresholds
top_vars = ranking_dif.head(10)['variavel'].tolist()
thresholds_top = thresholds[thresholds['variavel'].isin(top_vars)].sort_values(by='f1_score', ascending=False)

print('\nMétricas de Thresholds para Top Variáveis por Diferença de Média:')
display(thresholds_top)

# Merge para análise combinada
merged = pd.merge(thresholds, estatisticas[['variavel', 'diferenca_media']], on='variavel')

# Normalizar f1 e diferença de média para criar score combinado
merged['f1_score_norm'] = (merged['f1_score'] - merged['f1_score'].min()) / (merged['f1_score'].max() - merged['f1_score'].min())
merged['dif_media_norm'] = (merged['diferenca_media'] - merged['diferenca_media'].min()) / (merged['diferenca_media'].max() - merged['diferenca_media'].min())
merged['score_combinado'] = 0.6 * merged['f1_score_norm'] + 0.4 * merged['dif_media_norm']
merged['legenda'] = merged['variavel'].map(legendas_dict)

# Plotar score combinado
ranking_final = merged.sort_values(by='score_combinado', ascending=False)
plt.figure(figsize=(10,6))
sns.barplot(data=ranking_final.head(10), y='legenda', x='score_combinado', palette='Greens_d')
plt.title('Top 10 Variáveis por Score Combinado (0.6 F1 + 0.4 Diferença Média)')
plt.xlabel('Score Combinado')
plt.ylabel(None)
plt.grid(True, axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig(f'{base_output}/Imagens/Modelo_02/top10_score_combinado.png')
plt.close()

# Exibir tabela final
print("\nRanking Final das Variáveis (Score Combinado):")
display(ranking_final[['variavel', 'legenda', 'diferenca_media', 'f1_score', 'score_combinado']].head(10))


Métricas de Thresholds para Top Variáveis por Diferença de Média:


C:\Users\julia\AppData\Local\Temp\ipykernel_76052\2193272291.py:6: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=ranking_dif.head(10), y='legenda', x='diferenca_media', palette='Blues_d')


,variavel,threshold,precision,recall,f1_score,legenda
0,precipitation_probability_55km,25.0,0.635674,0.720430,0.675403,Probabilidade de Chuva (%) em uma Raio de 55km
1,precipitation_probability_35km,25.0,0.625078,0.714695,0.666890,Probabilidade de Chuva (%) em uma Raio de 35km
2,lwe_precipitation_smooth_rate_maximum_20km,0.4,0.638255,0.681720,0.659272,Tx. de Precipitacao (mm/h) Màx. em um Raio de ...
3,lwe_precipitation_smooth_rate_maximum_35km,0.5,0.547317,0.804301,0.651379,Tx. de Precipitacao (mm/h) Màx. em um Raio de ...
4,precipitation_probability_20km,15.0,0.604824,0.701075,0.649402,Probabilidade de Chuva (%) em uma Raio de 20km
5,lwe_precipitation_smooth_rate_maximum_55km,1.1,0.580980,0.722581,0.644089,Tx. de Precipitacao (mm/h) Màx. em um Raio de ...
6,lwe_precipitation_smooth_rate_maximum_10km,0.0,0.588756,0.668100,0.625923,Tx. de Precipitacao (mm/h) Màx. em um Raio de ...
7,precipitation_probability_10km,0.0,0.588756,0.668100,0.625923,Probabilidade de Chuva (%) em uma Raio de 10km
8,lwe_precipitation_smooth_rate,0.0,0.690710,0.453047,0.547186,Tx. de Precipitacao (mm/h)
9,lwe_precipitation_smooth_rate_minimum_10km,0.0,0.811441,0.274552,0.410284,Tx. de Precipitacao (mm/h) Mín. em um Raio de ...



Ranking Final das Variáveis (Score Combinado):


C:\Users\julia\AppData\Local\Temp\ipykernel_76052\2193272291.py:34: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=ranking_final.head(10), y='legenda', x='score_combinado', palette='Greens_d')


,variavel,legenda,diferenca_media,f1_score,score_combinado
1,precipitation_probability_35km,Probabilidade de Chuva (%) em uma Raio de 35km,44.546171,0.666890,0.991352
0,precipitation_probability_55km,Probabilidade de Chuva (%) em uma Raio de 55km,41.989307,0.675403,0.977031
4,precipitation_probability_20km,Probabilidade de Chuva (%) em uma Raio de 20km,42.559276,0.649402,0.955739
7,precipitation_probability_10km,Probabilidade de Chuva (%) em uma Raio de 10km,40.517416,0.625923,0.913546
3,lwe_precipitation_smooth_rate_maximum_35km,Tx. de Precipitacao (mm/h) Màx. em um Raio de ...,3.332108,0.651379,0.605363
5,lwe_precipitation_smooth_rate_maximum_55km,Tx. de Precipitacao (mm/h) Màx. em um Raio de ...,3.929179,0.644089,0.603322
2,lwe_precipitation_smooth_rate_maximum_20km,Tx. de Precipitacao (mm/h) Màx. em um Raio de ...,2.098708,0.659272,0.602302
6,lwe_precipitation_smooth_rate_maximum_10km,Tx. de Precipitacao (mm/h) Màx. em um Raio de ...,1.358527,0.625923,0.561776
8,lwe_precipitation_smooth_rate,Tx. de Precipitacao (mm/h),0.618638,0.547186,0.475145
9,lwe_precipitation_smooth_rate_minimum_10km,Tx. de Precipitacao (mm/h) Mín. em um Raio de ...,0.286791,0.410284,0.333094


In [19]:
print("""\n
Conclusão:
- As variáveis mais assertivas para criação de cenários de decisão são:
  1. precipitation_probability_35km
  2. precipitation_probability_55km
  3. precipitation_probability_20km

- Essas três variáveis lideram o ranking combinado (estatística + desempenho preditivo) e apresentam excelente equilíbrio entre separação de classes (parada vs não parada) e eficácia prática (F1-score).

- Variáveis de taxa de precipitação como 'lwe_precipitation_maximum_35km' ou 'maximum_20km' possuem utilidade secundária — sendo úteis em composições mais conservadoras — mas ficam atrás em performance geral.

- Com base nisso, recomenda-se utilizar essas probabilidades como base principal para cenários conservador, moderado e arrojado, ajustando os thresholds conforme o apetite ao risco operacional.
""")



Conclusão:
- As variáveis mais assertivas para criação de cenários de decisão são:
  1. precipitation_probability_35km
  2. precipitation_probability_55km
  3. precipitation_probability_20km

- Essas três variáveis lideram o ranking combinado (estatística + desempenho preditivo) e apresentam excelente equilíbrio entre separação de classes (parada vs não parada) e eficácia prática (F1-score).

- Variáveis de taxa de precipitação como 'lwe_precipitation_maximum_35km' ou 'maximum_20km' possuem utilidade secundária — sendo úteis em composições mais conservadoras — mas ficam atrás em performance geral.

- Com base nisso, recomenda-se utilizar essas probabilidades como base principal para cenários conservador, moderado e arrojado, ajustando os thresholds conforme o apetite ao risco operacional.



In [20]:
# Gerar cenários fixos com variáveis escolhidas manualmente
cenarios = {
    'conservador': [('precipitation_probability_55km', 15.0)],
    'moderado': [('precipitation_probability_35km', 25.0), ('lwe_precipitation_smooth_rate_maximum_35km', 0.5)],
    'arrojado': [('precipitation_probability_20km', 30.0), ('lwe_precipitation_smooth_rate_maximum_20km', 0.4)]
}

modelador = RainScenarioModel(
    df=df_unificado,
    cenarios=cenarios,
    base_output=base_output,
    legendas_dict=legendas_dict,
    modelo_nome="Modelo_02"  # ou "Modelo_01", "Modelo_RF", etc.
)

modelador.aplicar_cenarios()


Cenário: CONSERVADOR
Variáveis usadas:
[('precipitation_probability_55km', 15.0)]
Precision: 0.57 | Recall: 0.80 | F1 Score: 0.67

Cenário: MODERADO
Variáveis usadas:
[('precipitation_probability_35km', 25.0), ('lwe_precipitation_smooth_rate_maximum_35km', 0.5)]
Precision: 0.64 | Recall: 0.71 | F1 Score: 0.67

Cenário: ARROJADO
Variáveis usadas:
[('precipitation_probability_20km', 30.0), ('lwe_precipitation_smooth_rate_maximum_20km', 0.4)]
Precision: 0.67 | Recall: 0.60 | F1 Score: 0.63
